In [3]:
from db_communication import db_communication
import numpy as np
import pandas as pd
from datetime import datetime

### Init tables/ dataframes

In [4]:
config = {
    'user': 'root',
    'password': 'FDS-apm1',
    'host': 'min-ifm-xdm.ad.fh-bielefeld.de',
    'port': '3306',
    'database': 'Fahrradshop' 
}
my_db = db_communication(config)

Successfully connected.


In [5]:
cluster_df = my_db.get_table('Merkmalcluster')

In [6]:
parts_df = my_db.get_table('Einzelteile')

In [7]:
features_df = my_db.get_table('Merkmale')

In [9]:
config_df = my_db.get_table('Konfiguration')
config_df.index = pd.MultiIndex.from_arrays([config_df['AuftragNr'],config_df['KonfigNr']])
config_df = config_df['MerkmalNr'].to_frame()
config_df.head(5)

MerkmalNr
AuftragNr KonfigNr           
1         100          100008
          101          200001
          102          300006
          103          400005
          104          500001

In [ ]:
parts_df['Bezeichnung']

In [ ]:
features_df.transpose().loc['Bezeichnung']

In [ ]:
cluster_df.head(5)

### Direct demand matrix

In [ ]:
# generate Multiindex for the columns
col_main = cluster_df['Merkmalsbezeichnung'][features_df['Merkmalcluster'].values-1]
col_main.name = 'Cluster'
col_sub = features_df['Bezeichnung']
col_sub.name = 'Merkmal'
col = pd.MultiIndex.from_arrays([col_main,col_sub])

In [ ]:
# part name as index 
index_names = parts_df['Bezeichnung']
index_names.name = 'Einzelteil'

In [ ]:
# fill data with NaNs
data = np.ones((len(index_names),len(col))) * np.nan

df = pd.DataFrame(data=data,columns=col,index=index_names)
df

In [ ]:
# save template as Excel with current datetime as version
#df.to_excel('DirectDemandMatrix_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx')

In [ ]:
features_df[features_df['Merkmalcluster'] == 7]

### Verify direct demand matrix

In [11]:
# read excel
filename = 'DirectDemandMatrix_20200614_110337.xlsx'
directDemand_df = pd.read_excel(filename,
                                header=[0,1],
                                index_col=0)

In [ ]:
directDemand_df

In [ ]:
# check columns
print("Columns with NaNs only:\n")
for col_name,col in directDemand_df.iteritems():
    if col.isnull().all():
        print(col_name)

In [ ]:
# check rows
print("Rows with NaNs only:\n")
for row_name,row in directDemand_df.iterrows():
    if row.isnull().all():
        print(row_name)

In [12]:
def checkConfig(orderId,
                features_df=features_df,cluster_df=cluster_df,
                config_df=config_df, directDemand_df=directDemand_df):
    """
    verification through test configurations
    """
    # init lists
    feature_names = []
    part_names = []
    part_count = []
    for row_name,row in config_df.loc[orderId].iterrows():
        # map feature id to feature name and cluster
        feature_i = features_df.set_index('MerkmalNr').loc[row.values[0]]
        feature_name = feature_i['Bezeichnung']
        # get current cluster name
        feature_cluster = feature_i['Merkmalcluster']
        cluster_name = cluster_df.set_index('ClusterNr').loc[str(feature_cluster)].iloc[0]

        # get current parts per feature
        parts = directDemand_df[cluster_name,feature_name].dropna()
        if len(parts) != 0:
            part_names.append(parts.index.to_list())
        else:
            part_names.append([None])
        part_count.append(parts.values.astype(int))
        feature_names.append([feature_name for i in range(len(part_names[-1]))])

    # flatten lists for multiindexing
    feature_names = [feature_i for feature_name in feature_names for feature_i in feature_name] 
    part_names = [part_i for part_name in part_names for part_i in part_name] 

    # to df
    index = pd.MultiIndex.from_arrays([feature_names,part_names])
    index.names=['Merkmal','Einzelteil']
    df = pd.DataFrame(index=index,columns=['Anzahl'])
    # fill data
    i = 0
    for part_i in part_count:
        if len(part_i) != 0:
            for j in part_i:
                df.iloc[i] = [j]
                i += 1
        else:
            df.iloc[i] = [None]
            i += 1

    return df

In [17]:
checkConfig(5)

Anzahl
Merkmal                             Einzelteil                                               
Damen Trekkingbike                  Radon Scart Light Lady                                  1
                                    Cube Universal Fahrradstaender                          1
                                    Red Cycling Products Front Tray silver                  1
                                    SKS Velo 55 Trekking Schutzbleche                       1
Gelaendefederung Extrem             Marzocchi Bomber 58 Federgabel                          1
Aventurinrot II metallic            NaN                                                  None
Rennradlenker ohne Klingel          Ritchey Comp Streem III Di2 Road Lenker                 1
                                    Cube Lenkerband Kork                                    2
Felgen Gelaende Standard            Mavic Crossmax 29" Laufradsatz Intl                     2
                                    Ryde Zac 19 DH-C3003 Nabendynamo silver                 1
Bereifung Gelaende                  WTB Vigilante Faltreifen 27,5x2,5" TCS Light Fa...      2
                                    NoTubes Universal Tubeless Ventil 35mm MTB              2
                                    SCHWALBE No.13 Fahrradschlauch                          2
Gelaende Kettenschaltung mechanisch XX1 Eagle DUB                                           1
                                    XX1 Eagle Triggershifter                                1
                                    X-Horizon, X-Actuation, X-Sync, Roller Bearing ...      1
                                    XX1 Eagle Kette                                         1
                                    XG-1295 Eagle Kassette                                  1
                                    Direct Mount, X-Sync                                    1
                                    Horn Catena A08/48 Kettenschutz                         1
Trekkingsattel Damen                Red Cycling Products City Comfort Saddle Women          1
Felgenbremse Standard               Shimano Alivio BR-T4000 HR V-Brake                      1
                                    Shimano Alivio BR-T4000 VR V-Brake                      1
                                    Shimano BL-T4000 Bremshebel                             2
                                    Clarks CP200 Bremsbeläge                                4
                                    Clarks Stainless Steel Brake Cable                      2
Komfort Batteriebeleuchtung         Supernova Airstream 2 Frontlicht                        1
                                    Supernova Airstream Tail Light 2                        1
                                    Leuchtstreifen                                          2
Komfort Pedale                      Moto Reflex Pedale                                      2